First, Select GPU in the runtime session.

Change environment:

In [ ]:
!sed -i 's/np.complex/np.complex128/g' /usr/local/lib/python3.10/dist-packages/librosa/core/constantq.py

In [ ]:
!sed -i 's/np.float/np.float64/g' /usr/local/lib/python3.10/dist-packages/librosa/util/utils.py

import necessary requirements:

In [ ]:
%pip install --upgrade pip
%pip uninstall -y tensorflow numba
%pip install matplotlib\
numpy==1.19.3 \
inflect==0.2.5 \
librosa==0.6.0 \
Unidecode==1.0.22 \
pillow
%pip install janome==0.4.2
# %pip install scipy==1.0.0
%pip install denoiser==0.1.0
%pip install librosa==0.8.0
%pip install numba
%pip install pysoundfile==0.9.0.post1
%pip install unidecode==1.3.4
%pip install pyopenjtalk==0.2.0
%pip install inflect==5.6.2
%pip install janome==0.4.2



Clone repo from github:

In [ ]:
!git clone https://github.com/CjangCjengh/tacotron2-japanese.git
!cp -r /content/tacotron2-japanese/* /content

Clone trained model:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/gdrive/MyDrive/tacotron2-japanese/* /content

In [ ]:
!pip install pyopenjtalk

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from waveglow.denoiser import Denoiser

In [ ]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower',
                       interpolation='none')

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050

In [ ]:
checkpoint_path = "ayachi_nene_2"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval()#.half()

In [ ]:
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval()#.half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [ ]:
text = "こんにちは。あやちねねです。どうぞよろしくお願いいたします。"
sequence = np.array(text_to_sequence(text, ['japanese_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

In [ ]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)